# ===============================================
# 🏆 Project: The "Log-Sentinel" Swarm
# ===============================================
### **Track: Enterprise Agents 🖥️**

### **1. 🚀 The Pitch (Enterprise Agents Track)**

**The Problem:** In any large-scale IT enterprise, servers produce thousands of log lines every second. It is impossible for a human "SysAdmin" (System Administrator) to monitor this live stream for critical events. A single "FATAL" error, a slow-down in performance, or a "Failed Login" from an attacker could go unnoticed for hours, leading to system crashes or security breaches.

**The Solution:** "The 'Log-Sentinel' Swarm" is an autonomous **enterprise agent** team that acts as a 24/7 "daemon." It monitors a (mocked) live stream of server logs.

When a new log line is detected, the swarm activates:
1.  A **Sequential** `PlannerAgent` logs and pre-parses the log line.
2.  A **Parallel Team** of specialist agents (`SecurityAgent`, `PerformanceAgent`, `ErrorAgent`) investigate the line simultaneously for different issues.
3.  The `SecurityAgent` **Reads from Long-Term Memory** to check the log's IP against a `known_bad_ips` list.
4.  The `SecurityAgent` also **Writes to Long-Term Memory** to track `failed_login_tracker`. This is a **stateful** feature: if it sees 5 failed logins from the *same* IP in 60 seconds, it escalates the alert to "CRITICAL - BRUTE FORCE ATTACK".
5.  An **Evaluation Agent** (the `SysAdminAgent`) acts as the "brain." It reads all the technical reports, *evaluates* the combined risk, and *generates* a single, human-readable alert with a priority level.
6.  A final agent uses a **Real Custom Tool** to write all "CRITICAL" alerts to an `CRITICAL_ALERTS.md` file for the on-call engineer.

This project automates a high-stakes, real-world IT operations workflow, enabling instant threat detection and response.

### **2. ⚙️ Features Demonstrated**

This notebook is a capstone project that demonstrates all 6 key concepts:

1.  **Multi-agent System:**
    * **Sequential Agents:** `PlannerAgent` -> (Parallel Team) -> `SysAdminAgent` -> `PublisherAgent`.
    * **Parallel Agents:** `SecurityAgent`, `PerformanceAgent`, and `ErrorAgent` all run at the same time.
    * **Loop Agent:** The main `run_log_sentinel_swarm` function acts as a "daemon" loop, processing a queue of log lines.

2.  **Tools:**
    * **Mocked Built-in Tool:** `tool_lookup_ip_info` is used by the `SecurityAgent` (simulates a Geo-IP database lookup).
    * **Real Custom Tool:** `tool_save_alert_to_file` saves the final, formatted "CRITICAL" alert to a `.md` file.

3.  **Long-Running Operations (Pause/Resume):**
    * The main swarm function is a **Loop** that processes the log queue and then "pauses" (`time.sleep`) to wait for the next log line.

4.  **Long-Term Memory (Stateful & Complex):**
    * This project uses an advanced, stateful memory system.
    * **Static Read:** The `SecurityAgent` **reads** from `MEMORY_BANK["known_bad_ips"]` (a pre-loaded set).
    * **Stateful Read/Write:** The `SecurityAgent` **reads AND writes** to `MEMORY_BANK["failed_login_tracker"]` (a dictionary of lists) to track login failures over time and detect a "brute force" attack. This is the most complex and powerful feature.

5.  **Agent Evaluation (The "Brain"):**
    * The `SysAdminAgent` is the "brain." It reads and evaluates three different technical reports, *calculates* a `priority_level`, and *generates* a new, human-readable `justification` string for its final decision.

6.  **Observability:**
    * A `log_message()` function provides a rich, real-time log of every agent's actions and decisions.

### **3. 🤖 System Architecture**

This swarm operates on a Sequential -> Parallel -> Sequential workflow.


Here is the step-by-step data and logic flow:

1.  **[LOOP START]** (`run_log_sentinel_swarm`)
    * `PlannerAgent` pulls one `log_line` from the (mocked) queue.

2.  **[PARALLEL TEAM]** (Specialists)
    * The `PlannerAgent` passes the `log_line` to three specialists who run at the *same time*:
        * `SecurityAgent`: Uses Regex to find IPs. Checks `MEMORY_BANK["known_bad_ips"]` (Memory **Read**). If "Failed Login", it **reads AND writes** to `MEMORY_BANK["failed_login_tracker"]` (Stateful Memory).
        * `PerformanceAgent`: Uses Regex to find `response_time_ms=...` and checks if it's over a threshold.
        * `ErrorAgent`: Uses Regex to find keywords like `FATAL` or `ERROR`.

3.  **[SEQUENTIAL 2]** (`SysAdminAgent` - The "Brain")
    * Waits for all three parallel agents to finish.
    * **Evaluates** their reports, calculates a final `priority_level` (e.g., "FATAL" is `PRIORITY_1`).
    * **Generates** a `justification` string (e.g., "CRITICAL: Brute force attack detected from IP 1.2.3.4").
    * **Writes** this finding to `MEMORY_BANK["decision_log"]` (Memory **Write**).

4.  **[SEQUENTIAL 3]** (`PublisherAgent`)
    * **If** the "Brain's" priority is `PRIORITY_1`, this agent uses the **Custom Tool** (`tool_save_alert_to_file`) to save the justification to `CRITICAL_ALERTS.md`.

5.  **[LOOP END]**
    * The swarm **pauses** (`time.sleep`) and waits for the next log line.

### **4. 📚 Table of Contents**

* **Cell 1:** Imports & Libraries
* **Cell 2:** Global Setup (Memory Bank & Logger)
* **Cell 3:** Pre-load Memory (Known Bad IPs)
* **Cell 4:** Agent Toolbox (Mocked & Custom Tools)
* **Cell 5:** The Agent Team: Part 1 (Specialists)
* **Cell 6:** The Agent Team: Part 2 (The "Brain")
* **Cell 7:** The "Daemon" Loop (Main Function)
* **Cell 8:** 🚀 **Run the Swarm!** 🚀
* **Cell 9:** Review: Long-Term Memory (The Stateful Tracker)
* **Cell 10:** 🔔 **Review: The Final "Critical Alerts" File** 🔔

### **Cell 1: 📚 Imports & Libraries**

In [1]:
import time
import json
import datetime
import random
import os
import textwrap # Used for formatting our final Markdown report
import re # Regular Expressions, for parsing log lines
from collections import defaultdict

print("Cell 1: Libraries imported successfully.")

Cell 1: Libraries imported successfully.


### **Cell 2: 🧠 Global Setup (Memory Bank & Logger)**

In [2]:
MEMORY_BANK = {
    "known_bad_ips": set(),
    # This is our complex, stateful memory.
    # It will hold: {"ip_address": [timestamp1, timestamp2, ...]}
    "failed_login_tracker": defaultdict(list),
    # This list will hold the "Brain's" audit log
    "decision_log": [],
    "alerts_fired": 0,
}

# This helper function will act as our Observability/Logging
def log_message(agent_name, message, level="INFO"):
    """Helper function for formatted logging."""
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"[{timestamp}] [{agent_name}] [{level}]: {message}")

print("Cell 2: Global Memory Bank and Logger initialized.")

Cell 2: Global Memory Bank and Logger initialized.


### **Cell 3: 💾 Pre-load Memory (Known Bad IPs)**
We pre-load the Long-Term Memory with known attacker IPs for the `SecurityAgent` to **read** from.

In [3]:
MEMORY_BANK["known_bad_ips"].add("198.51.100.10")
MEMORY_BANK["known_bad_ips"].add("203.0.113.5")

print("Cell 3: Long-Term Memory pre-loaded with 'Known Bad IPs'.")
print(f"Known bad IPs: {MEMORY_BANK['known_bad_ips']}")

Cell 3: Long-Term Memory pre-loaded with 'Known Bad IPs'.
Known bad IPs: {'198.51.100.10', '203.0.113.5'}


### **Cell 4: 🛠️ Agent Toolbox (Mocked & Custom Tools)**

In [4]:
# --- Mocked Built-in Tool (Geo-IP Lookup) ---
def tool_lookup_ip_info(ip_address: str) -> str:
    """MOCK BUILT-IN TOOL: Simulates a database lookup for IP info."""
    log_message("GeoIPTool", f"Looking up info for: {ip_address}")
    
    # Mock database
    db = {
        "198.51.100.10": {"country": "Russia", "is_known_attacker": True},
        "10.1.1.5": {"country": "USA", "is_known_attacker": False},
        "172.16.0.4": {"country": "USA", "is_known_attacker": False},
        "1.2.3.4": {"country": "China", "is_known_attacker": True}
    }
    
    info = db.get(ip_address, {"country": "Unknown", "is_known_attacker": False})
    return json.dumps(info)

# --- REAL Custom Tool (Markdown File Saver) ---
def tool_save_alert_to_file(priority: str, justification: str, filename: str):
    """
    REAL CUSTOM TOOL: Saves the final, formatted alert to a Markdown file
    for the on-call engineer.
    """
    log_message("CustomFileTool", f"Saving {priority} alert to '{filename}'")
    try:
        # textwrap.dedent removes leading whitespace, making our report clean
        report = textwrap.dedent(f"""
        -------------------------------------------
        ### 🚨 {priority} ALERT
        **Justification:** {justification}
        **Timestamp:** {datetime.datetime.now().isoformat()}
        -------------------------------------------
        """)
        
        with open(filename, 'a') as f:
            f.write(report + "\n")
            
        log_message("CustomFileTool", "Alert successfully saved.")
        MEMORY_BANK["alerts_fired"] += 1
    except Exception as e:
        log_message("CustomFileTool", f"Failed to save file: {e}", level="ERROR")

print("Cell 4: All tools (mocked and custom) are defined.")

Cell 4: All tools (mocked and custom) are defined.


### **Cell 5: 🤖 The Agent Team: Part 1 (The Specialists)**
These are the parallel agents who perform the first-pass investigation.

In [5]:
# --- Sequential Agent 1: The Planner ---
def planner_agent(log_line: str) -> dict:
    """SEQUENTIAL AGENT 1: Logs the new line and does basic parsing."""
    log_message("PlannerAgent", f"New log line received: {log_line[:70]}...")
    
    # Use Regex to find the IP address, if it exists
    ip_match = re.search(r'ip=(\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})', log_line)
    ip = ip_match.group(1) if ip_match else None
    
    return {"log_line": log_line, "ip": ip}

# --- Parallel Agent 1: The Security Agent (Complex & Stateful) ---
def security_agent(log_line: str, ip: str) -> dict:
    """PARALLEL AGENT 1: Checks for security threats."""
    log_message("SecurityAgent", "Analyzing for security threats...")
    flags = []
    
    if not ip:
        log_message("SecurityAgent", "Security review complete. No IP found.")
        return {"security_flags": flags}

    # 1. STATIC READ from Long-Term Memory
    if ip in MEMORY_BANK["known_bad_ips"]:
        flags.append(f"CRITICAL: Log from KNOWN ATTACKER IP: {ip}")
        # Use tool to get more info
        info = json.loads(tool_lookup_ip_info(ip))
        flags.append(f"Info: IP is from {info['country']}")

    # 2. STATEFUL READ/WRITE to Long-Term Memory (Brute Force Detection)
    if "Failed Login" in log_line:
        now = datetime.datetime.now()
        tracker = MEMORY_BANK["failed_login_tracker"]
        
        # WRITE to memory
        tracker[ip].append(now)
        
        # READ from memory (stateful check)
        # Get all failed logins from this IP in the last 60 seconds
        recent_failures = [t for t in tracker[ip] 
                           if (now - t).total_seconds() < 60]
        
        if len(recent_failures) >= 5:
            flags.append(f"CRITICAL: BRUTE FORCE ATTACK detected from IP {ip} ({len(recent_failures)} failures in 60s)")
        else:
            flags.append(f"WARNING: Failed Login from {ip}. Total failures from this IP: {len(recent_failures)}.")
    
    log_message("SecurityAgent", f"Security review complete. {len(flags)} flags found.")
    return {"security_flags": flags}

# --- Parallel Agent 2: The Performance Agent ---
def performance_agent(log_line: str) -> dict:
    """PARALLEL AGENT 2: Checks for performance degradation."""
    log_message("PerformanceAgent", "Analyzing for performance issues...")
    flags = []
    
    # Use Regex to find response_time_ms=...
    match = re.search(r'response_time_ms=(\d+)', log_line)
    if match:
        time_ms = int(match.group(1))
        if time_ms > 1000:
            flags.append(f"PERFORMANCE: High response time detected: {time_ms}ms")
            
    log_message("PerformanceAgent", f"Performance review complete. {len(flags)} flags found.")
    return {"performance_flags": flags}

# --- Parallel Agent 3: The Error Agent ---
def error_agent(log_line: str) -> dict:
    """PARALLEL AGENT 3: Checks for critical system errors."""
    log_message("ErrorAgent", "Analyzing for system errors...")
    flags = []
    
    # Use Regex to find keywords
    if re.search(r'(FATAL|CRITICAL_ERROR|SEGFAULT)', log_line):
        flags.append(f"CRITICAL: Fatal system error detected: {log_line}")
        
    elif re.search(r'ERROR', log_line):
        flags.append(f"ERROR: Non-fatal error detected: {log_line}")

    log_message("ErrorAgent", f"Error review complete. {len(flags)} flags found.")
    return {"error_flags": flags}

print("Cell 5: Planner and Parallel Specialist Agents defined.")

Cell 5: Planner and Parallel Specialist Agents defined.


### **Cell 6: 🧠 The Agent Team: Part 2 (The "Brain")**
This is the Evaluation Agent that synthesizes the reports and the Publisher Agent that saves the file.

In [6]:
# --- Evaluation Agent: The "Brain" ---
def sys_admin_agent(log_line: str, security_report: dict, performance_report: dict, error_report: dict) -> dict:
    """
    EVALUATION AGENT: The "brain."
    Synthesizes reports from the parallel team into one decision.
    Demonstrates: Agent Evaluation, Generative AI (Mocked)
    """
    log_message("SysAdminAgent", f"Synthesizing all reports for log line...")
    
    all_flags = (security_report["security_flags"] + 
                 performance_report["performance_flags"] + 
                 error_report["error_flags"])
    
    # --- MOCKED GENERATIVE AI (Evaluation Logic) ---
    priority_level = "INFO"
    justification = ""
    
    if not all_flags:
        priority_level = "INFO"
        justification = "All systems normal."
    else:
        # Generate justification
        justification = f"Alerts found: [{'; '.join(all_flags)}]"
        
        # Determine priority
        if any("CRITICAL" in flag for flag in all_flags):
            priority_level = "PRIORITY_1_CRITICAL"
        elif any("ERROR" in flag for flag in all_flags):
            priority_level = "PRIORITY_2_ERROR"
        elif any("PERFORMANCE" in flag for flag in all_flags):
            priority_level = "PRIORITY_3_WARNING"
        elif any("WARNING" in flag for flag in all_flags):
            priority_level = "PRIORITY_3_WARNING"
            
    log_message("SysAdminAgent", f"Synthesis complete. Priority: {priority_level}")
    
    # WRITE the final decision to the audit log (Long-Term Memory)
    MEMORY_BANK["decision_log"].append({
        "log_line": log_line,
        "priority": priority_level,
        "justification": justification,
        "timestamp": datetime.datetime.now().isoformat()
    })
    
    return {"final_priority": priority_level, "justification": justification}


# --- Sequential Agent 3: The Publisher ---
def publisher_agent(brain_decision: dict):
    """SEQUENTIAL AGENT 3: Uses the Custom Tool to save the final report if critical."""
    
    if brain_decision["final_priority"] == "PRIORITY_1_CRITICAL":
        log_message("PublisherAgent", f"Decision is CRITICAL. Calling Custom Tool...")
        
        # Use the REAL Custom Tool
        tool_save_alert_to_file(
            priority=brain_decision["final_priority"],
            justification=brain_decision["justification"],
            filename="CRITICAL_ALERTS.md"
        )
    else:
        log_message("PublisherAgent", f"Decision is {brain_decision['final_priority']}. No file needed.")

print("Cell 6: Evaluation ('Brain') and Publisher Agents defined.")

Cell 6: Evaluation ('Brain') and Publisher Agents defined.


### **Cell 7: 🔄 The "Daemon" Loop (Main Function)**
This is the main "Loop Agent" that simulates a server, processing a queue of new log lines.

In [7]:
def run_log_sentinel_swarm(loop_delay_seconds: int):
    """
    Main function to run the agent swarm (Loop Agent).
    Simulates a "daemon" processing a queue of log lines.
    Demonstrates: Loop, Long-Running Ops (Pause/Resume), Sequential, Parallel
    """
    
    log_message("LogSentinelSwarm", "Swarm is active. Monitoring log stream...")
    
    # --- MOCKED LOG QUEUE ---
    # This simulates a live stream of server logs.
    # Note how they are designed to test every single rule.
    mocked_log_queue = [
        # 1. A normal, clean log line
        "[2025-11-26 19:00:00] [INFO] [web-server-1] [ip=10.1.1.5] Request /index.html status=200 response_time_ms=50",
        
        # 2. A FATAL error
        "[2025-11-26 19:00:01] [FATAL] [db-server-3] [ip=10.1.1.8] CRITICAL_ERROR: Database connection pool exhausted. SEGFAULT.",
        
        # 3. A high response time
        "[2025-11-26 19:00:02] [WARN] [web-server-2] [ip=10.1.1.6] Request /api/v1/data status=200 response_time_ms=2500",
        
        # 4. A "Failed Login" from a *new* IP (to start the Brute Force tracker)
        "[2025-11-26 19:00:03] [WARN] [auth-server-1] [ip=1.2.3.4] Failed Login for user 'admin'",
        
        # 5. A login from a *known* bad IP
        "[2025-11-26 19:00:04] [WARN] [auth-server-1] [ip=198.51.100.10] Failed Login for user 'root'",
        
        # 6-9. Four more failures from the *same* new IP (to trigger the tracker)
        "[2025-11-26 19:00:05] [WARN] [auth-server-1] [ip=1.2.3.4] Failed Login for user 'admin'",
        "[2025-11-26 19:00:06] [WARN] [auth-server-1] [ip=1.2.3.4] Failed Login for user 'admin'",
        "[2025-11-26 19:00:07] [WARN] [auth-server-1] [ip=1.2.3.4] Failed Login for user 'admin'",
        "[2025-11-26 19:00:08] [WARN] [auth-server-1] [ip=1.2.3.4] Failed Login for user 'admin'", # This is the 5th one!
    ]
    
    # --- Clear alert file at the start ---
    alert_filename = "CRITICAL_ALERTS.md"
    if os.path.exists(alert_filename):
        os.remove(alert_filename)
        log_message("LogSentinelSwarm", "Cleared old critical alerts file.")

    # --- Phase 1: The "Daemon" (Loop) ---
    loop_count = 0
    while mocked_log_queue:
        loop_count += 1
        log_message("LogSentinelSwarm", f"--- Starting Loop {loop_count} ---")
        
        # Get the next item from the queue
        log_line = mocked_log_queue.pop(0)
        
        # --- Phase 2: Planning (Sequential) ---
        planned_task = planner_agent(log_line)
        
        # --- Phase 3: Parallel Execution ---
        log_message("PlannerAgent", f"Spawning parallel agents for log...")
        
        report_part_1 = security_agent(planned_task["log_line"], planned_task["ip"])
        report_part_2 = performance_agent(planned_task["log_line"])
        report_part_3 = error_agent(planned_task["log_line"])
        
        # --- Phase 4: Evaluation (Sequential "Brain") ---
        brain_decision = sys_admin_agent(
            log_line=planned_task["log_line"],
            security_report=report_part_1,
            performance_report=report_part_2,
            error_report=report_part_3
        )
        
        # --- Phase 5: Publishing (Custom Tool) ---
        publisher_agent(brain_decision)
        
        # --- Phase 6: LONG-RUNNING OPERATION (PAUSE/RESUME) ---
        if mocked_log_queue:
            log_message("LogSentinelSwarm", f"Operation complete. Pausing for {loop_delay_seconds}s...")
            time.sleep(loop_delay_seconds)
            log_message("LogSentinelSwarm", "Resuming operation. Checking for new logs...")
    
    log_message("LogSentinelSwarm", "All logs processed. Swarm is shutting down.")

print("Cell 7: Main 'Daemon' loop function defined.")

Cell 7: Main 'Daemon' loop function defined.


### **Cell 8: 🚀 Run the Swarm!**
This cell will execute the entire swarm. You will see the logs print out in real-time as it processes all nine (mocked) log lines.

In [8]:
# --- Configuration ---\
LOOP_DELAY_SECONDS = 1 # How long to "pause" (1s for demo)

# --- Clear the memory from previous runs (but keep the known bad IPs!) ---
MEMORY_BANK["failed_login_tracker"] = defaultdict(list)
MEMORY_BANK["decision_log"] = []
MEMORY_BANK["alerts_fired"] = 0
print("--- Cleared Memory Bank history for new run ---")

# --- Run the entire agent swarm ---\
run_log_sentinel_swarm(
    loop_delay_seconds=LOOP_DELAY_SECONDS
)

print("\n\n" + "="*40)
print("✅ SWARM RUN COMPLETE ✅")
print(f"Total Critical Alerts Generated: {MEMORY_BANK['alerts_fired']}")
print("="*40)

--- Cleared Memory Bank history for new run ---
[2025-11-26 14:28:15] [LogSentinelSwarm] [INFO]: Swarm is active. Monitoring log stream...
[2025-11-26 14:28:15] [LogSentinelSwarm] [INFO]: --- Starting Loop 1 ---
[2025-11-26 14:28:15] [PlannerAgent] [INFO]: New log line received: [2025-11-26 19:00:00] [INFO] [web-server-1] [ip=10.1.1.5] Request /ind...
[2025-11-26 14:28:15] [PlannerAgent] [INFO]: Spawning parallel agents for log...
[2025-11-26 14:28:15] [SecurityAgent] [INFO]: Analyzing for security threats...
[2025-11-26 14:28:15] [SecurityAgent] [INFO]: Security review complete. 0 flags found.
[2025-11-26 14:28:15] [PerformanceAgent] [INFO]: Analyzing for performance issues...
[2025-11-26 14:28:15] [PerformanceAgent] [INFO]: Performance review complete. 0 flags found.
[2025-11-26 14:28:15] [ErrorAgent] [INFO]: Analyzing for system errors...
[2025-11-26 14:28:15] [ErrorAgent] [INFO]: Error review complete. 0 flags found.
[2025-11-26 14:28:15] [SysAdminAgent] [INFO]: Synthesizing all re

### **Cell 9: 📊 Review: Long-Term Memory (The Stateful Tracker)**
Let's inspect the `MEMORY_BANK`. This proves our agents were **reading and writing** to memory. You will see the `decision_log` from the "Brain" and the stateful `failed_login_tracker` built by the `SecurityAgent`.

In [9]:
print("Inspecting Long-Term Memory...")
print(f"\n--- Stateful Failed Login Tracker (Written by 'SecurityAgent') ---")
print(f"IP '1.2.3.4' (Brute Force IP) was logged {len(MEMORY_BANK['failed_login_tracker']['1.2.3.4'])} times.")
print(f"IP '198.51.100.10' (Known Attacker) was logged {len(MEMORY_BANK['failed_login_tracker']['198.51.100.10'])} times.")

print(f"\n--- Decision Log (Written by 'SysAdminAgent' Brain) ---")
print(f"Total decisions logged: {len(MEMORY_BANK['decision_log'])}")
# We only print the justifications for clarity
justifications = [d['justification'] for d in MEMORY_BANK['decision_log']]
print(json.dumps(justifications, indent=2))

Inspecting Long-Term Memory...

--- Stateful Failed Login Tracker (Written by 'SecurityAgent') ---
IP '1.2.3.4' (Brute Force IP) was logged 5 times.
IP '198.51.100.10' (Known Attacker) was logged 1 times.

--- Decision Log (Written by 'SysAdminAgent' Brain) ---
Total decisions logged: 9
[
  "All systems normal.",
  "Alerts found: [CRITICAL: Fatal system error detected: [2025-11-26 19:00:01] [FATAL] [db-server-3] [ip=10.1.1.8] CRITICAL_ERROR: Database connection pool exhausted. SEGFAULT.]",
  "Alerts found: [PERFORMANCE: High response time detected: 2500ms]",
  "Alerts found: [WARNING: Failed Login from 1.2.3.4. Total failures from this IP: 1.]",
  "Alerts found: [CRITICAL: Log from KNOWN ATTACKER IP: 198.51.100.10; Info: IP is from Russia; WARNING: Failed Login from 198.51.100.10. Total failures from this IP: 1.]",
  "Alerts found: [WARNING: Failed Login from 1.2.3.4. Total failures from this IP: 2.]",
  "Alerts found: [WARNING: Failed Login from 1.2.3.4. Total failures from this IP: 3

# 🔔 Review: The Final "Critical Alerts" File
We will print the contents of the CRITICAL_ALERTS.md file that was created by our Custom Tool. This is the file the on-call engineer would be paged for

In [10]:
print("Inspecting 'CRITICAL_ALERTS.md' file (created by Custom Tool)...")
print("="*80 + "\n")

alert_filename = "CRITICAL_ALERTS.md"
try:
    with open(alert_filename, 'r') as f:
        print(f.read())
except FileNotFoundError:
    print(f"--- No file found. (This is OK if no critical alerts were flagged) ---")

Inspecting 'CRITICAL_ALERTS.md' file (created by Custom Tool)...


-------------------------------------------
### 🚨 PRIORITY_1_CRITICAL ALERT
**Justification:** Alerts found: [CRITICAL: Fatal system error detected: [2025-11-26 19:00:01] [FATAL] [db-server-3] [ip=10.1.1.8] CRITICAL_ERROR: Database connection pool exhausted. SEGFAULT.]
**Timestamp:** 2025-11-26T14:28:16.099875
-------------------------------------------


-------------------------------------------
### 🚨 PRIORITY_1_CRITICAL ALERT
**Justification:** Alerts found: [CRITICAL: Log from KNOWN ATTACKER IP: 198.51.100.10; Info: IP is from Russia; WARNING: Failed Login from 198.51.100.10. Total failures from this IP: 1.]
**Timestamp:** 2025-11-26T14:28:19.101975
-------------------------------------------


-------------------------------------------
### 🚨 PRIORITY_1_CRITICAL ALERT
**Justification:** Alerts found: [CRITICAL: BRUTE FORCE ATTACK detected from IP 1.2.3.4 (5 failures in 60s)]
**Timestamp:** 2025-11-26T14:28:23.1045